## Libraries and settings

## Section 2: File Formats  (~7 min)

In [1]:
# Libraries
import os
import warnings
import pandas as pd
import numpy as np
import pyarrow

# Ignore warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/workspaces/data_engineer_assessment/part_2


### Task 2.1: Reading Different File Formats

The `part_2/` directory contains data in two formats:
- `apartments_data_winterthur.csv` — apartment rental data in CSV format
- `supermarkets.json` — supermarket locations from OpenStreetMap in JSON format

**Your tasks:**
1. Read the CSV file into a DataFrame using `pd.read_csv()` with appropriate parameters
2. Read the JSON file into a DataFrame using `pd.read_json()`
3. Display the first 3 rows and the shape of each DataFrame

In [26]:
# Task 2.1.1 — Read the CSV file
# TODO: Read 'apartments_data_winterthur.csv' into a DataFrame
df_apartments = pd.read_csv('apartments_data_winterthur.csv')

# Print info and first 3 rows
print(f'Info: {df_apartments.info()}')

df_apartments.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      120 non-null    object
 1   web-scraper-start-url  120 non-null    object
 2   rooms_area_price_raw   120 non-null    object
 3   address_raw            120 non-null    object
 4   price_raw              120 non-null    object
 5   description_raw        120 non-null    object
 6   text_raw               120 non-null    object
dtypes: object(7)
memory usage: 6.7+ KB
Info: None


,web-scraper-order,web-scraper-start-url,rooms_area_price_raw,address_raw,price_raw,description_raw,text_raw
0,1693993818-1,https://www.immoscout24.ch/de/wohnung/mieten/o...,"6,5 Zimmer, 143 m², CHF 3017.—","Am Eulachpark 25, 8404 Winterthur, ZH",CHF 3017.—,«Sie suchen die spezielle Maisonettewohnung?»,"6,5 Zimmer, 143 m², CHF 3017.—Am Eulachpark 25..."
1,1693993818-2,https://www.immoscout24.ch/de/wohnung/mieten/o...,"1 Zimmer, 132 m², CHF 3260.—","Katharina Sulzer Platz 2, 8400 Winterthur, ZH",CHF 3260.—,«In Loft-iger Höhe MIETEN OHNE KAUTION»,"1 Zimmer, 132 m², CHF 3260.—Katharina Sulzer P..."
2,1693993818-3,https://www.immoscout24.ch/de/wohnung/mieten/o...,"4,5 Zimmer, 117 m², CHF 3782.—","8400 Winterthur, ZH",CHF 3782.—,"«MÖBLIERT, TEMPORÄR: 4½ ZI-WOHNUNG IN WINTERTH...","4,5 Zimmer, 117 m², CHF 3782.—8400 Winterthur,..."


In [ ]:
# Task 2.1.2 — Read the JSON file
# TODO: Read 'supermarkets.json' into a DataFrame
df_supermarkets = pd.read_json('supermarkets.json')

# Print last 3 rows
df_supermarkets.tail(3)

,type,id,lat,lon,tags
3389,node,11107076347,47.466556,9.048250,"{'addr:city': 'Wil SG', 'addr:housenumber': '3..."
3390,node,11107594883,47.322228,8.529748,"{'addr:city': 'Adliswil', 'addr:housenumber': ..."
3391,node,11129298207,47.537518,7.608581,"{'brand': 'Coop', 'brand:wikidata': 'Q432564',..."


### Task 2.2: Nested Objects

The `df_supermarkets` DataFrame has a nested object in the `tags` column. Each row contains a dictionary with multiple OSM (OpenStreetMap) attributes like brand, opening hours, address details, etc.

**Your tasks:**
1. Inspect the `tags` column to understand its structure (display one example)
2. Flatten the nested `tags` dictionary into separate columns using `pd.json_normalize()`
3. Combine the flattened columns with the original location columns (`type`, `id`, `lat`, `lon`)
4. Drop the original `tags` column and display the resulting DataFrame
5. Compare the shape before and after flattening

In [4]:
# Task 2.2 Solution — Flattening Nested Objects

#TODO Step 1: Inspect the tags column structure
print("Step 1: Inspect one example of the nested 'tags' column")
print(f"Type of tags[0]: {type(df_supermarkets['tags'].iloc[0])}")
print(f"\nExample tags dictionary (first row):")
print(df_supermarkets['tags'].iloc[0])
print(f"\nNumber of keys in first row: {len(df_supermarkets['tags'].iloc[0])}")


Step 1: Inspect one example of the nested 'tags' column
Type of tags[0]: <class 'dict'>

Example tags dictionary (first row):
{'brand': 'Spar', 'brand:wikidata': 'Q610492', 'brand:wikipedia': 'en:SPAR (retailer)', 'name': 'Spar', 'opening_hours': 'Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00', 'shop': 'supermarket'}

Number of keys in first row: 6


In [5]:
# Step 2 & 3: Flatten the tags columns and combine with original columns

#TODO Flatten the tags dictionary into separate columns
tags_normalized = pd.json_normalize(df_supermarkets['tags'])

#TODO Combine with the original location columns
df_supermarkets_flattened = pd.concat(
    [df_supermarkets[['type', 'id', 'lat', 'lon']], tags_normalized],
    axis=1
)

print(f"Original shape: {df_supermarkets.shape}")
print(f"Flattened shape: {df_supermarkets_flattened.shape}")


Original shape: (3392, 5)
Flattened shape: (3392, 237)


In [6]:
#TODO Step 4: Display information about the flattened DataFrame
print("Column names:")
print(df_supermarkets_flattened.columns.tolist())
print(f"\nFirst 3 rows of flattened data:")
print(df_supermarkets_flattened.head(3))
print(f"\nData types:")
print(df_supermarkets_flattened.dtypes)


Column names:
['type', 'id', 'lat', 'lon', 'brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop', 'addr:city', 'addr:housenumber', 'addr:postcode', 'addr:street', 'alt_name', 'branch', 'cash_withdrawal', 'cash_withdrawal:fee', 'cash_withdrawal:purchase_required', 'cash_withdrawal:type', 'operator', 'website', 'wheelchair', 'addr:country', 'check_date', 'check_date:opening_hours', 'level', 'source:opening_hours', 'takeaway', 'phone', 'cash_withdrawal:operator', 'source', 'created_by', 'changing_table', 'organic', 'toilets', 'air_conditioning', 'fax', 'post_office', 'payment:cash', 'payment:maestro', 'toilets:wheelchair', 'addr:place', 'old_name', 'currency:CHF', 'payment:coins', 'payment:credit_cards', 'payment:debit_cards', 'payment:mastercard', 'payment:visa', 'opening_hours:covid19', 'internet_access', 'payment:american_express', 'payment:app', 'payment:apple_pay', 'payment:contactless', 'payment:google_pay', 'payment:postfinance_card', 'payment:samsung_pay', 

### Task 2.3: File Format Conversion

**Your tasks:**
1. Take the apartments DataFrame (from Task 2.1) and write it to a **Parquet** file
2. Read the Parquet file back and verify the data is identical
3. Compare the file sizes of the CSV and Parquet files

In [ ]:
# Task 2.3.1 — Write to Parquet
# TODO: Save df_apartments to 'apartments_winterthur.parquet'
df_apartments.to_parquet('apartments_winterthur.parquet', index=False)

# Task 2.3.2 — Read back from Parquet and verify
# TODO: Read the parquet file and compare shape/dtypes with the original
df_from_parquet = pd.read_parquet('apartments_winterthur.parquet')

print(f'Original shape:  {df_apartments.shape}')
print(f'Parquet shape:   {df_from_parquet.shape}')
print(f'DataFrames equal: {df_apartments.equals(df_from_parquet)}')

Original shape:  (120, 7)
Parquet shape:   (120, 7)
DataFrames equal: True


In [ ]:
# Task 2.3.3 — Compare file sizes
# TODO: Use os.path.getsize() to compare the CSV and Parquet file sizes
csv_size = os.path.getsize('apartments_data_winterthur.csv')
parquet_size = os.path.getsize('apartments_winterthur.parquet')

print(f'CSV file size:     {csv_size:>10,} bytes')
print(f'Parquet file size:  {parquet_size:>10,} bytes')
print(f'Compression ratio:  {csv_size / parquet_size:.2f}x')

CSV file size:         99,628 bytes
Parquet file size:      48,863 bytes
Compression ratio:  2.04x
